In [6]:
import numpy as np

def max_repeatedNaNs_v2(a):
    mask = np.concatenate(([False],np.isnan(a),[False]))
    print(mask)
    print(~mask)
    if ~mask.any():
        return 0
    else:
        idx = np.nonzero(mask[1:] != mask[:-1])[0]
        return (idx[1::2] - idx[::2]).max()
    
max_repeatedNaNs_v2(np.array([np.nan, np.nan, np.nan, 1, 2, 3, np.nan, np.nan]))

[False  True  True  True False False False  True  True False]
[ True False False False  True  True  True False False  True]


3

In [5]:
import pandas as pd

df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': ['apple', 'banana', 'orange', 'pear', 'peach'],
    'C': [True, False, True, False, True]
})

print(df['A'].loc[1:len(df)])

1    2
2    3
3    4
4    5
Name: A, dtype: int64


In [14]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

##### glassnode api
# insert your API key here
API_KEY = '2NdxyJCRiTcs5XZUidfNDW9olTb'

# set time to download
since = 1589155200 # 2020 May 11
# since = 1646092800 # 2022 Mar 1
until = 1677628800 # 2023 Mar 1
resolution = "1h"

res = requests.get("https://api.glassnode.com/v1/metrics/market/price_usd_close",
    params={"a": "BTC", "s": since, "u": until, "api_key": API_KEY, "i": resolution})
df_value = pd.read_json(res.text, convert_dates=['t'])

res = requests.get("https://api.glassnode.com/v1/metrics/market/price_usd_close",
    params={"a": "BTC", "s": since, "u": until, "api_key": API_KEY, "i": resolution})
df_price = pd.read_json(res.text, convert_dates=['t'])

df = pd.merge(df_value,df_price,how='inner',on='t')
df = df.rename(columns={'v_x':'value','v_y':'price'})

# this part is to import price data
# df = pd.read_csv('BTC-USD.csv')
# df = df[['Date','Close']]
df['chg'] = df['price'].pct_change()

def bband(window, threshold):
    df['ma'] = df['value'].rolling(window).mean()
    df['sd'] = df['value'].rolling(window).std()
    df['z'] = ( df['value'] - df['ma'] ) / df['sd']

    # for i in range(len(df)):
    #     if df.loc[i,'z'] > threshold:
    #         df.loc[i,'pos'] = 1
    #     elif df.loc[i,'z'] < -threshold:
    #         df.loc[i, 'pos'] = -1
    #     else:
    #         df.loc[i, 'pos'] = 0

    df['pos'] = np.where(df['z'] > threshold, 1, np.where(df['z'] < -threshold, -1, 0))

    df['pos_t-1'] = df['pos'].shift(1)
    df['trade'] = abs(df['pos'] - df['pos_t-1'])
    df['pnl'] = df['pos_t-1'] * df['chg'] - df['trade'] * 0.05/100 # 5 bps
    df['cumu'] = df['pnl'].cumsum()
    df['dd'] = df['cumu'].cummax() - df['cumu']

    df['bnh_pnl'] = df['chg']
    df.loc[0:window-1, 'bnh_pnl'] = 0
    df['bnh_cumu'] = df['bnh_pnl'].cumsum()

    annual_return = round(df['pnl'].mean() * 365 * 24, 3)
    sharpe = round(df['pnl'].mean() / df['pnl'].std() * np.sqrt(365 * 24), 3)
    mdd = round(df['dd'].max(), 3)
    calmar = round(annual_return / mdd, 3)

    avg_return = df.loc[window:len(df), 'pnl'].mean()
    return_sd = df.loc[window:len(df), 'pnl'].std()
    precise_sharpe = round(avg_return / return_sd * np.sqrt(365 * 24), 3)

    bnh_sharpe = round(df['bnh_pnl'].mean() / df['bnh_pnl'].std() * np.sqrt(365 * 24), 3)

    # print(df)
    print(window, threshold, 'annual return', annual_return, 'sharpe', sharpe, 'precise_sharpe', precise_sharpe, 'bnh_sharpe', bnh_sharpe, 'mdd', mdd, 'calmar', calmar)

    return pd.Series([window, threshold, precise_sharpe], index=['window', 'threshold', 'sharpe'])

# window_list = [10,20,30,40,50,60,70,80,90,100]
# threshold_list = [0,0.5,1,1.5,2]

window_list = np.arange(100,3000,100)
threshold_list = np.arange(0,2.5,0.25)

##### Optimization 區
### create a list
result_list = []

# for window in window_list:
#     for threshold in threshold_list:
#         result_list.append(bband(window, threshold))
#
# result_df = pd.DataFrame(result_list)
# result_df = result_df.sort_values(by='sharpe',ascending=False)
# print(result_df)
#
# data_table = result_df.pivot(index='window',columns='threshold',values='sharpe')
# sns.heatmap(data_table, annot=True, fmt='g', cmap='Greens')
# plt.show()

##### Backtest 區
# parameters
window = 1700
threshold = 0.5
bband(window, threshold)
# rsi(window, threshold)
fig = px.line(df, x='t', y=['cumu', 'bnh_cumu', 'dd'], title='strategy')
fig.show()

ValueError: Expected object or value

In [2]:
import itertools

# Suppose you have a dynamic number of lists
list1 = [1, 2, 3]
list2 = [4, 5]
list3 = [6, 7, 8]

# You can put all lists into a list of lists
all_lists = [list1, list2, list3]

# Use itertools.product() to get all possible combinations
combinations = list(itertools.product(*all_lists))

# Print the result
print(combinations)
print(*all_lists)

[(1, 4, 6), (1, 4, 7), (1, 4, 8), (1, 5, 6), (1, 5, 7), (1, 5, 8), (2, 4, 6), (2, 4, 7), (2, 4, 8), (2, 5, 6), (2, 5, 7), (2, 5, 8), (3, 4, 6), (3, 4, 7), (3, 4, 8), (3, 5, 6), (3, 5, 7), (3, 5, 8)]
[1, 2, 3] [4, 5] [6, 7, 8]


In [3]:
import inspect

class MyClass:
    def my_method(self, arg1, arg2):
        pass

# Get the method object
method_obj = MyClass.my_method

# Get the signature of the method object
sig = inspect.signature(method_obj)

# Print the number of arguments needed by the method
num_args = len(sig.parameters)
print(f"The method needs {num_args} arguments.")


The method needs 3 arguments.
